In [1]:
import numpy as np
from numpy.linalg.linalg import norm
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
#import tensorflow as tf
#from tensorflow.keras.models import Model
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
#from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, LayerNormalization
#from keras.optimizers import Adam
#from keras.optimizers import RMSprop
#from keras.regularizers import l2
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.markers import MarkerStyle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd


# データの読み込み
data = np.load('/home/maeda/data/bsiso_lee13/prepro_anomaly_7vals.npz')
print('data = ', data.files)

data =  ['lat', 'lon', 'time', 'real_time', 'olr', 'u850', 'v850', 'u200', 'v200', 'h850', 'pr_wtr']


In [16]:
lat = data['lat'][24:49]
lon = data['lon']
olr = data['olr'][80:-435,24:49,:]
u850 = data['u850'][80:-435,24:49,:]
v850 = data['v850'][80:-435,24:49,:]
u200 = data['u200'][80:-435,24:49,:]
v200 = data['v200'][80:-435,24:49,:]
h850 = data['h850'][80:-435,24:49,:]
pr_wtr = data['pr_wtr'][80:-435,24:49,:]
time = data['time'][80:-435]
real_time = pd.to_datetime(time, unit='h', origin=pd.Timestamp('1800-01-01')) # 時刻をdatetime型に変換
print(lat.shape, lon.shape, olr.shape, u850.shape, v850.shape, u200.shape, v200.shape, h850.shape, pr_wtr.shape)
print(real_time[0], real_time[-1])

(25,) (144,) (15556, 25, 144) (15556, 25, 144) (15556, 25, 144) (15556, 25, 144) (15556, 25, 144) (15556, 25, 144) (15556, 25, 144)
1979-03-22 00:00:00 2021-10-22 00:00:00


In [17]:
# 標準化処理
def normalization(data):
  data_mean = np.mean(data, axis=0)
  data_std  = np.std(data, axis=0)
  data_norm = (data - data_mean) / data_std
  print('Raw Data        = ', data.max(), data.min())
  print('Normalized Data = ', data_norm.max(), data_norm.min())
  del data_mean, data_std
  return data_norm

olr_norm  = normalization(olr)
u850_norm = normalization(u850)
v850_norm = normalization(v850)
u200_norm = normalization(u200)
v200_norm = normalization(v200)
h850_norm = normalization(h850)
pr_wtr_norm = normalization(pr_wtr)

Raw Data        =  125.99925062995904 -191.72708324469755
Normalized Data =  7.704964842602574 -19.955042052030905
Raw Data        =  36.59931389543901 -33.440710581927625
Normalized Data =  8.445019621765717 -7.323080941264589
Raw Data        =  33.573279469116336 -29.869047367771834
Normalized Data =  11.417965337901181 -10.35929392602082
Raw Data        =  59.14801304108833 -71.19605653246393
Normalized Data =  7.687482060373113 -7.13136258235884
Raw Data        =  68.2906863765084 -71.56800375378678
Normalized Data =  7.676351093927562 -9.64070628852694
Raw Data        =  155.87304853426116 -299.0225253344866
Normalized Data =  5.565155482458696 -11.301119272354802
Raw Data        =  46.15640080135331 -42.05674743351691
Normalized Data =  8.603196998553319 -7.0538494118246735


In [19]:
# bsiso index (eEOF) 読み込み
data_file = '/home/maeda/archive/kikuchi_bsiso_index.dat'

year    = np.loadtxt(data_file, usecols = 0, dtype = int)
month   = np.loadtxt(data_file, usecols = 1, dtype = int)
day     = np.loadtxt(data_file, usecols = 2, dtype = int)
PC     = np.loadtxt(data_file, usecols = [3, 4])
#PCy     = np.loadtxt(data_file, usecols = 4)
phase   = np.loadtxt(data_file, usecols = 5, dtype = int)
Amp_nrm_bsiso = np.loadtxt(data_file, usecols = 6, dtype = float)
#index = np.loadtxt(data_file)

print('month = ', month.shape)
print('phase = ', phase.shape)
print( 'PC = ', PC.shape)
print('Amp_nrm_bsiso = ', Amp_nrm_bsiso.shape)
print(year, month, day)

month =  (15556,)
phase =  (15556,)
PC =  (15556, 2)
Amp_nrm_bsiso =  (15556,)
[1979 1979 1979 ... 2021 2021 2021] [ 3  3  3 ... 10 10 10] [22 23 24 ... 20 21 22]


In [20]:
# 全て一律にずらしたあと、インデクシングする
lead_time = 0
multi_forcast = False
if multi_forcast == True:
  output_shape = 2 * (lead_time + 1)
else:
  output_shape = 2
print('output shape = ', output_shape)

amp = Amp_nrm_bsiso[10:-lead_time-1]
ph = phase[10:-lead_time-1]
yy = year[10:-lead_time-1]
mm = month[10:-lead_time-1]
dd = day[10:-lead_time-1]
rt = real_time[10:-lead_time-1]
# 教師データは前進させる
if multi_forcast == True:
  sup_data = []
  for j in range(lead_time+1):
    sup_j = PC[10+j:-lead_time-1+j]
    sup_data.append(sup_j)
  sup_data = np.array(sup_data).transpose(1,0,2).reshape(-1, output_shape)
else:
  sup_data = PC[10+lead_time:]
print(sup_data.shape)


# 入力データの前処理
def preprocess(data):
  ipt_lag0  = data[10:-lead_time-1]
  ipt_lag5  = data[5:-lead_time-6]
  ipt_lag10 = data[:-lead_time-11]

  # =========
  # 訓練データの作成(通年データとする)
  idx = np.where((yy <= 2014))[0]
  ipt_lag0_train = ipt_lag0[idx]
  ipt_lag5_train = ipt_lag5[idx]
  ipt_lag10_train = ipt_lag10[idx]
  #ipt_train = np.concatenate([ipt_lag0_train, ipt_lag5_train, ipt_lag10_train], 1)
  ipt_train = np.stack([ipt_lag0_train, ipt_lag5_train, ipt_lag10_train], 3)

  # 検証データの作成
  idx = np.where((yy > 2014))[0]
  ipt_lag0_test = ipt_lag0[idx]
  ipt_lag5_test = ipt_lag5[idx]
  ipt_lag10_test = ipt_lag10[idx]
  #ipt_test = np.concatenate([ipt_lag0_test, ipt_lag5_test, ipt_lag10_test], 1)
  ipt_test = np.stack([ipt_lag0_test, ipt_lag5_test, ipt_lag10_test], 3)
  return ipt_train, ipt_test

olr_ipt_train, olr_ipt_test = preprocess(olr_norm)
u850_ipt_train, u850_ipt_test = preprocess(u850_norm)
v850_ipt_train, v850_ipt_test = preprocess(v850_norm)
ipt_train = np.concatenate([olr_ipt_train, u850_ipt_train, v850_ipt_train], 3)
ipt_test = np.concatenate([olr_ipt_test, u850_ipt_test, v850_ipt_test], 3)
#ipt_train, ipt_test = v850_ipt_train, v850_ipt_test

# その他のインデクシング
idx = np.where((yy <= 2014))[0]
sup_train = sup_data[idx]
idx = np.where((yy > 2014))[0]
sup_test = sup_data[idx]
amp = amp[idx]
ph = ph[idx]
yy = yy[idx]
mm = mm[idx]
dd = dd[idx]
rt = rt[idx]
print(sup_test.shape, ipt_test.shape, ipt_train.shape, ph.shape, yy.shape, mm.shape, dd.shape)
del olr_ipt_train, olr_ipt_test, u850_ipt_train, u850_ipt_test, v850_ipt_train, v850_ipt_test

output shape =  2
(15546, 2)
(2486, 2) (2486, 25, 144, 9) (13059, 25, 144, 9) (2486,) (2486,) (2486,) (2486,)
